In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp -r "/content/drive/MyDrive/PROJECT_FakeNewsDetection-KLKS/CodeKLKS/PipelineV2" /content/PipelineV2
!cp "/content/drive/MyDrive/PROJECT_FakeNewsDetection-KLKS/fakenewsdatasetv1.csv" /content/

!pip install -q emoji transformers


In [5]:
%cd /content
!python -m PipelineV2.run_pipeline --data_path /content/fakenewsdatasetv1.csv --output_dir /content/outputs

/content

[*] Data path:   /content/fakenewsdatasetv1.csv
[*] Output dir:  /content/outputs
[*] Seed:        42
[*] Min words:   8
[*] Near-dup thr:0.92

PIPELINE V2 - LOAD DATA
  Loaded: 1843 samples

  Cleaning text (unified pipeline)...
  After filter (>=8 words, >=10 chars): 1843 (removed 0)
  After exact dedup: 1843 (removed 0)

  Dataset: 1843 samples | REAL=974 (52.8%) | FAKE=869 (47.2%)

BUILDING NEAR-DUP CLUSTERS
  Groups: 1810 | Largest group size: 8
  Top 5 group sizes:
group
35     8
136    7
26     4
67     2
163    2

LEAK-SAFE SPLIT (STRATIFIED + GROUP-AWARE)
  Picked fold 1 for TEST: size=184 (0.100)
  Picked fold 5 for VAL: size=182 (0.110)
  Train:  1477 | REAL=777 (52.6%) | FAKE=700 (47.4%) | groups=1449
  Val  :   182 | REAL=98 (53.8%) | FAKE=84 (46.2%) | groups=182
  Test :   184 | REAL=99 (53.8%) | FAKE=85 (46.2%) | groups=179

  Group overlap: Train&Val=0 | Train&Test=0 | Val&Test=0

LEAK REPORT

  -- Exact Leak Check --
  Exact leak Train&Val: 0
  Exact leak Tra

In [6]:
!python -m PipelineV2.train_phobert \
    --data_path /content/fakenewsdatasetv1.csv \
    --output_dir /content/outputs \
    --epochs 5 \
    --batch_size 8

PHOBERT FINE-TUNING FOR FAKE NEWS DETECTION
  Device: cuda
  Model:  vinai/phobert-base
  Epochs: 5
  Batch:  8
  LR:     2e-05

PIPELINE V2 - LOAD DATA
  Loaded: 1843 samples

  Cleaning text (unified pipeline)...
  After filter (>=8 words, >=10 chars): 1843 (removed 0)
  After exact dedup: 1843 (removed 0)

  Dataset: 1843 samples | REAL=974 (52.8%) | FAKE=869 (47.2%)

BUILDING NEAR-DUP CLUSTERS
  Groups: 1810 | Largest group size: 8
  Top 5 group sizes:
group
35     8
136    7
26     4
67     2
163    2

LEAK-SAFE SPLIT (STRATIFIED + GROUP-AWARE)
  Picked fold 1 for TEST: size=184 (0.100)
  Picked fold 5 for VAL: size=182 (0.110)
  Train:  1477 | REAL=777 (52.6%) | FAKE=700 (47.4%) | groups=1449
  Val  :   182 | REAL=98 (53.8%) | FAKE=84 (46.2%) | groups=182
  Test :   184 | REAL=99 (53.8%) | FAKE=85 (46.2%) | groups=179

  Group overlap: Train&Val=0 | Train&Test=0 | Val&Test=0

LEAK REPORT

  -- Exact Leak Check --
  Exact leak Train&Val: 0
  Exact leak Train&Test: 0
  Exact leak V

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "vinai/phobert-base"
CKPT_PATH  = "/content/outputs/phobert_model/best_model_state_dict.pt"

id2label = {0: "REAL", 1: "FAKE"}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

state_dict = torch.load(CKPT_PATH, map_location=device)
missing, unexpected = model.load_state_dict(state_dict, strict=False)
print("Missing keys:", missing[:10], "..." if len(missing) > 10 else "")
print("Unexpected keys:", unexpected[:10], "..." if len(unexpected) > 10 else "")

model.eval()

@torch.no_grad()
def predict_text(text: str, max_length: int = 256):
    enc = tokenizer(text, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    enc = {k: v.to(device) for k, v in enc.items()}

    logits = model(**enc).logits
    probs = torch.softmax(logits, dim=-1).squeeze(0)
    pred_id = int(torch.argmax(probs).item())

    return {
        "label": id2label[pred_id],
        "prob_real": float(probs[0].item()),
        "prob_fake": float(probs[1].item()),
    }

while True:
    text = input("\nNhập tin (q để thoát): ").strip()
    if text.lower() in ["q", "quit", "exit"]:
        break
    if not text:
        print("Bạn chưa nhập gì.")
        continue
    out = predict_text(text)
    print(f"→ {out['label']} | P(REAL)={out['prob_real']:.4f} | P(FAKE)={out['prob_fake']:.4f}")


Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: vinai/phobert-base
Key                             | Status     | 
--------------------------------+------------+-
roberta.pooler.dense.bias       | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
roberta.pooler.dense.weight     | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.decoder.weight          | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.decoder.bias            | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
classifier.out_proj.weight      | MISSING    | 
classifier.dense.weight         | MISSING    | 
classifier.dense.bias           | MISSING    | 
classifier.out_proj.bias        | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initia

Missing keys: [] 
Unexpected keys: [] 

Nhập tin (q để thoát): Chủ tịch nước Cộng hòa xã hội chủ nghĩa Việt Nam hiện tại (nhiệm kỳ 2021-2026) là Đại tướng Lương Cường
→ FAKE | P(REAL)=0.0005 | P(FAKE)=0.9995

Nhập tin (q để thoát): Chủ tịch nước Cộng hòa xã hội chủ nghĩa Việt Nam hiện tại (nhiệm kỳ 2021-2026) là Tô Lâm
→ FAKE | P(REAL)=0.0005 | P(FAKE)=0.9995

Nhập tin (q để thoát): Theo báo Tuổi Trẻ ngày 12/1, Bộ Giáo dục cho biết kỳ thi tốt nghiệp THPT năm nay vẫn giữ nguyên hình thức như năm trước.
→ FAKE | P(REAL)=0.3024 | P(FAKE)=0.6976

Nhập tin (q để thoát): Messi có 6 quả bóng vàng
→ FAKE | P(REAL)=0.0006 | P(FAKE)=0.9994

Nhập tin (q để thoát): Messi có 5 quả bóng vàng
→ FAKE | P(REAL)=0.0006 | P(FAKE)=0.9994

Nhập tin (q để thoát): Messi có 4 quả bóng vàng
→ FAKE | P(REAL)=0.0006 | P(FAKE)=0.9994


KeyboardInterrupt: Interrupted by user